In [1]:
import tensorflow as tf
from tensorflow import keras

import os
from os import path

import tempfile
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
pd.options.mode.chained_assignment = None

import seaborn as sns

import sklearn

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-v5xjtvue because the default path (/home/juliuswa/installed/apache-tomcat-9.0.30/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


### Set DataFrame name:

In [2]:
pca_prec = 95

df_name = f"{pca_prec}"
print(f"Name: {df_name}")

data_path = f"../../RoboDocData/"

df_path = f"{data_path}/{df_name}"

if path.exists(df_path) == False:
    os.makedirs(df_path)

Name: 95


In [3]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [4]:
raw_df = pd.read_csv("../db/admissions.csv")
raw_df.head()

,patient_id,hadm_id,icustay_id,hospstay_seq,total_hospstays,length_of_stay_hospital,icustay_seq,number_of_icu_stays,length_of_stay_icu,total_length_of_stay_icu,...,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max,symptoms,patient_history,accident_causes,diagnoses
0,9973,100020,282580,5,8,10.61,1,1.0,1.10,1.101088,...,126.0,131.0,11.0,16.0,5.6,5.6,NaN,V1302,NaN,5990;34409;2761;5601;5849;0417;4280;59654;340;...
1,77221,100024,201668,1,1,6.33,1,1.0,1.08,1.079884,...,135.0,140.0,18.0,18.0,10.4,12.3,NaN,V4960,E8782,41401;4111;99662;5119;5180;4019;42731;45182;27401
2,9889,100068,204988,5,5,15.08,1,1.0,4.50,4.500347,...,128.0,132.0,39.0,43.0,15.0,15.1,78906,V1254;V4511;V4579;V8801;V5861,E8782,41401;5856;4111;40391;9971;4260;4592;5121;4107...
3,26885,100074,270174,1,1,2.73,1,1.0,2.79,2.787488,...,122.0,132.0,45.0,64.0,18.9,19.6,78552;79989,NaN,NaN,0389;51881;55220;5570;99592;07054;5715
4,14245,100102,236711,2,6,2.79,1,1.0,0.99,0.992882,...,134.0,136.0,27.0,29.0,9.5,11.5,7847,V4582;V4502,NaN,41401;42822;41402;4280;30029;30981;42731;40390...


## Clean null values:

In [5]:
raw_df.describe()

,patient_id,hadm_id,icustay_id,hospstay_seq,total_hospstays,length_of_stay_hospital,icustay_seq,number_of_icu_stays,length_of_stay_icu,total_length_of_stay_icu,...,inr_min,inr_max,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max
count,49276.000000,49276.000000,49276.000000,49276.000000,49276.000000,49276.000000,49276.0,48916.000000,49276.000000,48916.000000,...,43675.000000,43675.000000,43673.000000,43673.000000,48931.000000,48931.000000,48903.000000,48903.000000,48646.000000,48646.000000
mean,36830.670387,149981.859830,250083.991152,1.475627,1.951356,10.001152,1.0,1.073902,4.066593,5.389209,...,1.358426,1.647879,14.815815,16.837765,136.572357,140.137473,23.590986,28.013087,10.420413,13.649568
std,29043.766195,28915.694377,28907.221643,1.617056,2.672499,10.824429,0.0,0.305319,5.985749,11.561777,...,0.648030,1.398121,5.034032,9.430014,4.967009,4.581295,19.664591,22.627591,8.032359,11.289151
min,3.000000,100001.000000,200001.000000,1.000000,1.000000,0.000000,1.0,1.000000,0.000000,0.000347,...,0.500000,0.500000,2.390000,8.000000,1.210000,97.000000,1.000000,1.000000,0.100000,0.100000
25%,13300.500000,124911.750000,225039.750000,1.000000,1.000000,4.040000,1.0,1.000000,1.200000,1.253953,...,1.100000,1.100000,12.700000,13.200000,134.000000,138.000000,12.000000,14.000000,6.800000,8.800000
50%,26709.500000,150055.500000,250130.500000,1.000000,1.000000,6.920000,1.0,1.000000,2.120000,2.251157,...,1.200000,1.300000,13.600000,14.400000,137.000000,140.000000,17.000000,20.000000,9.300000,12.100000
75%,60230.250000,175055.500000,275154.750000,1.000000,2.000000,11.950000,1.0,1.000000,4.110000,4.911823,...,1.400000,1.600000,15.000000,16.700000,139.000000,142.000000,28.000000,33.000000,12.600000,16.400000
max,99999.000000,199999.000000,299999.000000,41.000000,41.000000,294.660000,1.0,7.000000,153.930000,568.103333,...,26.700000,48.800000,150.000000,150.000000,178.000000,182.000000,254.000000,272.000000,575.800000,846.700000


In [6]:
def get_sparse_columns(raw_df, limit):
    print(f"Under {limit * 100}% filled:")
    sparse_columns = []

    for c in raw_df.columns:
        rate = (sum(raw_df[c] == raw_df[c]) / len(raw_df))

    if rate < limit:
        print(f"\t{c}: {rate:.2f}")
        sparse_columns.append(c)

    return sparse_columns

def fill_nan_with_neighbor_mean(df, column):
    df[column].fillna(df[column].mean(), inplace=True)

    return df

def clean_df(raw_df, limit=0.2):
    # copy dataframe
    df = raw_df.copy()

    # pop standard columns
    df.pop('patient_id')
    df.pop('hadm_id')
    df.pop('icustay_id')
    df.pop('hospstay_seq')
    df.pop('icustay_seq')
    df.pop('total_hospstays')
    df.pop('number_of_icu_stays')
    df.pop('length_of_stay_icu')
    df.pop('total_length_of_stay_icu')

    if(len(df) > 1):
        # pop sparse columns
        sparse_columns = get_sparse_columns(raw_df, limit)

        for c in sparse_columns:
            df.pop(c)

        # pop sparse rows
            df.dropna(thresh=12, inplace=True)
            

    # clean data
    df = df.astype({"age": float,
                            "weight": float,
                            "glucose_max": float,
                            "glucose_mean": float})

    df.age[df.age > 90] = 90
    df.gender[df.gender == 'M'] = 1
    df.gender[df.gender == 'F'] = 0
    df.weight[df.weight > 300] = 300
    df.glucose_max[df.glucose_max > 1000] = 1000
    df.glucose_mean[df.glucose_mean > 1000] = 1000
    df.died_in_hospital[df.died_in_hospital == 't'] = 1
    df.died_in_hospital[df.died_in_hospital == 'f'] = 0

    # extract diseases
    symptoms_raw = df.symptoms
    df.pop('symptoms')
    diagnoses_raw = df.diagnoses
    df.pop('diagnoses')
    patient_history_raw = df.patient_history
    df.pop('patient_history')
    accident_causes_raw = df.accident_causes
    df.pop('accident_causes')
    
    print(df.columns)
    
    # fill nan values
    print("fill nan values")
    dont_fill = ["died_in_hospital",
                 "length_of_stay_hospital",
                 "days_to_death"]
    
    # temporarily extract dont_fill columns
    print("temporarily extract dont_fill columns")
    labels = {}
    for c in dont_fill:
        labels[c] = df.pop(c)          
    
    # fill NaNs
    print("fill NaNs")
    for c in df.columns:
        fill_nan_with_neighbor_mean(df, c)
    
    # normalize data
    print("normalize data")
    scaler = StandardScaler()
    M = scaler.fit_transform(df.values)
    
    """
    for i in range(10):
        no_nans = M[i] == M[i]
        M_compare = M[:, no_nans]
        M_no_nan = M_compare(np.all(M_compare == M_compare, axis=1))
        
        diff = M[:, no_nans] - M[i, no_nans]
        dists = np.linalg.norm(diff, axis=1)
        
    """
    
    print("reconvert to daterframe")
    print(M.shape)
    df = pd.DataFrame(M, columns=df.columns)
        
    print("readd columns")
    for k in labels:
        df[k] = labels[k]
    

    return df, symptoms_raw, diagnoses_raw, patient_history_raw, accident_causes_raw

In [7]:
df, symptoms_raw, diagnoses_raw, patient_history_raw, accident_causes_raw = clean_df(raw_df, 0.2)

Under 20.0% filled:
Index(['length_of_stay_hospital', 'days_to_death', 'died_in_hospital', 'age',
       'gender', 'weight', 'height', 'heartrate_mean', 'heartrate_min',
       'heartrate_max', 'meanbp_mean', 'meanbp_min', 'meanbp_max',
       'resprate_mean', 'resprate_min', 'resprate_max', 'tempc_mean',
       'tempc_min', 'tempc_max', 'spo2_mean', 'spo2_min', 'spo2_max',
       'glucose_mean', 'glucose_min', 'glucose_max', 'received_dialysis',
       'received_ventilation', 'urineoutput', 'mingcs', 'gcsmotor',
       'gcsverbal', 'gcseyes', 'aniongap_min', 'aniongap_max', 'albumin_min',
       'albumin_max', 'bands_min', 'bands_max', 'bicarbonate_min',
       'bicarbonate_max', 'bilirubin_min', 'bilirubin_max', 'creatinine_min',
       'creatinine_max', 'chloride_min', 'chloride_max', 'hematocrit_min',
       'hematocrit_max', 'hemoglobin_min', 'hemoglobin_max', 'lactate_min',
       'lactate_max', 'platelet_min', 'platelet_max', 'potassium_min',
       'potassium_max', 'ptt_min', '

In [8]:
df.head()

,age,gender,weight,height,heartrate_mean,heartrate_min,heartrate_max,meanbp_mean,meanbp_min,meanbp_max,...,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max,died_in_hospital,length_of_stay_hospital,days_to_death
0,-0.314227,0.882781,0.709301,5.792595e-01,-1.494342,-1.465284,-1.130373,0.793605,-0.182336,1.786196,...,4.001880e-16,-2.136028,-2.001557,-0.642731,-0.532931,-0.604004,-0.717645,0,10.61,95.0
1,0.435565,0.882781,0.478289,5.792595e-01,-0.465844,-0.262882,-0.790772,-0.181336,0.478330,-0.339928,...,-2.633326e-01,-0.317677,-0.030113,-0.285403,-0.444206,-0.002558,-0.120318,0,6.33,NaN
2,0.608594,-1.132784,-0.815376,-1.125754e+00,0.112687,0.338319,-0.499686,-0.092705,-0.035521,-0.223958,...,2.435605e-01,-1.731950,-1.782508,0.786583,0.664857,0.573828,0.129311,0,15.08,23.0
3,-0.487256,-1.132784,-0.122341,-7.847512e-01,1.141184,1.273521,0.373572,-0.801754,-0.108928,-0.378585,...,8.518322e-01,-2.944184,-1.782508,1.092864,1.596470,1.062503,0.530501,1,2.73,2.0
4,0.839300,0.882781,1.725752,-3.230626e-15,-1.237217,-1.264884,-1.178887,-1.333540,-1.430261,-1.035750,...,-3.534470e-01,-0.519716,-0.906311,0.174020,0.043782,-0.115329,-0.191641,0,2.79,489.0


In [9]:
df.describe()

,age,gender,weight,height,heartrate_mean,heartrate_min,heartrate_max,meanbp_mean,meanbp_min,meanbp_max,...,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max,length_of_stay_hospital,days_to_death
count,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,...,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,49276.000000,21266.000000
mean,1.105266e-16,1.018388e-16,-8.096634e-17,-3.217538e-15,-1.362657e-16,-6.618620e-17,2.555162e-16,1.205970e-15,-1.349679e-16,-3.887898e-16,...,-4.811702e-16,3.957113e-16,-6.321548e-15,9.716294e-15,1.029203e-16,-4.554807e-17,-1.058402e-16,-5.980550e-17,10.001152,516.919261
std,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,...,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,10.824429,750.269258
min,-2.851985e+00,-1.132784e+00,-3.679919e+00,-7.150134e+00,-3.551337e+00,-4.715110e+00,-3.556091e+00,-5.410569e+00,-4.293148e+00,-3.161874e+00,...,-2.621954e+00,-9.955116e-01,-2.734847e+01,-9.449233e+00,-1.153200e+00,-1.198368e+00,-1.293161e+00,-1.207988e+00,0.000000,0.000000
25%,-6.602848e-01,-1.132784e+00,-6.305664e-01,-3.230626e-15,-7.229681e-01,-6.636825e-01,-6.937438e-01,-7.131228e-01,-5.493726e-01,-5.718688e-01,...,-4.253544e-01,-3.985041e-01,-5.197164e-01,-4.682120e-01,-5.916841e-01,-6.216557e-01,-4.536424e-01,-4.323549e-01,4.040000,18.000000
50%,8.950739e-02,8.827810e-01,0.000000e+00,-3.230626e-15,-8.015685e-02,-6.248149e-02,-1.115714e-01,-9.270538e-02,0.000000e+00,-1.853008e-01,...,-1.932450e-01,-2.182755e-01,8.640052e-02,-3.011340e-02,-3.364495e-01,-3.111181e-01,-1.278591e-01,-1.292337e-01,6.920000,149.000000
75%,7.816233e-01,8.827810e-01,3.858846e-01,-3.230626e-15,6.269355e-01,6.055197e-01,6.161441e-01,5.277120e-01,5.517378e-01,3.172375e-01,...,0.000000e+00,4.001880e-16,4.904785e-01,4.079852e-01,2.250665e-01,2.212320e-01,2.605749e-01,2.362947e-01,11.950000,722.750000
max,1.531416e+00,8.827810e-01,1.013457e+01,3.581620e+01,5.062333e+00,4.680327e+00,8.572500e+00,6.731886e+00,4.882772e+00,7.507402e+00,...,2.852503e+01,1.499978e+01,8.369999e+00,9.169957e+00,1.176167e+01,1.082387e+01,7.084279e+01,7.426918e+01,294.660000,4166.000000


## Examine the class label imbalance:

In [10]:
neg, pos = np.bincount(df['died_in_hospital'])
total = neg + pos
print(f'Examples:\n    Total: {total}\n    Positive: {pos} ({100 * pos / total:.2f}% of total)\n')

Examples:
    Total: 49276
    Positive: 5672 (11.51% of total)



## Convert diagnoses:

In [11]:
def load_diseases_data():
    symptoms = []
    history = []
    diagnoses = []

    with open('../db/symptoms_dict.csv', 'r') as f:
        for line in f:
            split = line.strip("\n").split(",")
            symptoms.append([split[0], split[1], split[2]])
            
    with open('../db/patient_history_dict.csv', 'r') as f:
        for line in f:
            split = line.strip("\n").split(",")
            history.append([split[0], split[1], split[2]])

    with open('../db/diagnoses_dict.csv', 'r') as f:
        for line in f:
            split = line.strip("\n").split(",")
            diagnoses.append([split[0], split[1], split[2]])
            
    return np.asarray(symptoms[1:]), np.asarray(history[1:]), np.asarray(diagnoses[1:])


def diseases_to_matrix(symptoms, patient_history, diagnoses, symptoms_dict, patient_history_dict, diagnoses_dict):
    Xs = np.zeros((len(diagnoses), len(symptoms_dict)))
    
    for i, ss in enumerate(symptoms):
        lst = str(ss).split(';')

        for s in lst:
            Xs[i][symptoms_dict == s] = 1

            
    Xph = np.zeros((len(diagnoses), len(patient_history_dict)))
    
    for i, phs in enumerate(patient_history):
        lst = str(phs).split(';')

        for ph in lst:
            Xph[i][patient_history_dict == ph] = 1


    Xd = np.zeros((len(diagnoses), len(diagnoses_dict)))
    
    for i, ds in enumerate(diagnoses):
        lst = str(ds).split(';')

        for d in lst:
            Xd[i][diagnoses_dict == d] = 1

    X = np.concatenate((Xs, Xph), axis=1)
    X = np.concatenate((X, Xd), axis=1)

    return X, Xs, Xph, Xd

In [12]:
print("admissions with symptoms: ", len(symptoms_raw[symptoms_raw == symptoms_raw]) / len(symptoms_raw))

admissions with symptoms:  0.5345198473902102


In [13]:
print("admissions with patient_history: ", len(patient_history_raw[patient_history_raw == patient_history_raw]) / len(patient_history_raw))

admissions with patient_history:  0.5195632762399546


In [14]:
print("admissions with diagnoses: ", len(diagnoses_raw[diagnoses_raw == diagnoses_raw]) / len(diagnoses_raw))

admissions with diagnoses:  0.9990258949590064


In [15]:
symptoms_dict, patient_history_dict, diagnoses_dict  = load_diseases_data()

print(f"symptoms: {len(symptoms_dict)}, patient_history: {len(patient_history_dict)}, diagnoses: {len(diagnoses_dict)}")

t_start = time.time()
disease_matrix, Xs, Xph, Xd = diseases_to_matrix(symptoms_raw, patient_history_raw, diagnoses_raw, symptoms_dict[:,0], patient_history_dict[:,0], diagnoses_dict[:,0])
t_end = time.time()

shape = disease_matrix.shape
ones = sum(sum(disease_matrix))

print(f"Time elapsed: {(t_end - t_start) / 60:.2f} minutes")
print(f"disease_matrix shape: {shape}")
print(f"percentage of ones: {(100 * ones / (shape[0] * shape[1])):.2f}%")

symptoms: 1141, patient_history: 470, diagnoses: 4730
Time elapsed: 0.36 minutes
disease_matrix shape: (49276, 6341)
percentage of ones: 0.18%


In [16]:
if True:
    with open(f"{data_path}/disease_matrix_raw.npy", 'wb') as f:
        np.save(f, disease_matrix)

    with open(f"{data_path}/symptoms_dict.npy", 'wb') as f:
        np.save(f, symptoms_dict)

    with open(f"{data_path}/patient_history_dict.npy", 'wb') as f:
        np.save(f, patient_history_dict)

    with open(f"{data_path}/diagnoses_dict.npy", 'wb') as f:
        np.save(f, diagnoses_dict)

### Reduce dimensionality of diagnoses with PCA:

In [17]:
from sklearn.decomposition import PCA
import pickle as pk

In [18]:
if pca_prec >= 100:
    pca_prec = int(pca_prec/100)
else:
    pca_prec = (1/100) * pca_prec
    

pca = PCA(n_components=pca_prec)

t_start = time.time()
M = pca.fit_transform(disease_matrix)
t_end = time.time()

with open(f"{df_path}/disease_matrix_{df_name}.npy", 'wb') as f:
    np.save(f, M)

pk.dump(pca, open(f"{df_path}/pca_{df_name}.pkl", 'wb')) 
# pca_reload = pk.load(open("pca.pkl",'rb')) 

print(f"Time elapsed: {(t_end - t_start) / 60:.2f} minutes")
print(M.shape)

Time elapsed: 5.05 minutes
(49276, 1438)


In [19]:
print(pca.explained_variance_ratio_)

[3.25376498e-02 3.07601326e-02 2.08225528e-02 ... 5.55436796e-05
 5.53776333e-05 5.53469820e-05]


In [20]:
with open(f"{df_path}/disease_matrix_{df_name}.npy", 'rb') as f:
    M = np.load(f)
print(M.shape)

(49276, 1438)


### Join DataFrame and diagnose matrix:

In [21]:
df_diagnoses = pd.DataFrame(M)
df = df.reset_index(drop=True)
full_df = df.join(df_diagnoses)

print(df.shape)
print(full_df.shape)
print(df.columns)

(49276, 68)
(49276, 1506)
Index(['age', 'gender', 'weight', 'height', 'heartrate_mean', 'heartrate_min',
       'heartrate_max', 'meanbp_mean', 'meanbp_min', 'meanbp_max',
       'resprate_mean', 'resprate_min', 'resprate_max', 'tempc_mean',
       'tempc_min', 'tempc_max', 'spo2_mean', 'spo2_min', 'spo2_max',
       'glucose_mean', 'glucose_min', 'glucose_max', 'received_dialysis',
       'received_ventilation', 'urineoutput', 'mingcs', 'gcsmotor',
       'gcsverbal', 'gcseyes', 'aniongap_min', 'aniongap_max', 'albumin_min',
       'albumin_max', 'bands_min', 'bands_max', 'bicarbonate_min',
       'bicarbonate_max', 'bilirubin_min', 'bilirubin_max', 'creatinine_min',
       'creatinine_max', 'chloride_min', 'chloride_max', 'hematocrit_min',
       'hematocrit_max', 'hemoglobin_min', 'hemoglobin_max', 'lactate_min',
       'lactate_max', 'platelet_min', 'platelet_max', 'potassium_min',
       'potassium_max', 'ptt_min', 'ptt_max', 'inr_min', 'inr_max', 'pt_min',
       'pt_max', 'sodiu

## Normalize data:

In [22]:
from sklearn import preprocessing

length_of_stay_hospital = full_df.pop("length_of_stay_hospital")
#length_of_stay_icu = full_df.pop("length_of_stay_icu")
days_to_death = full_df.pop("days_to_death")
died_in_hospital = full_df.pop("died_in_hospital")

X = full_df.values #returns a numpy array
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
normed_df = pd.DataFrame(X_scaled, columns=full_df.columns)

normed_df["length_of_stay_hospital"] = length_of_stay_hospital
#normed_df["length_of_stay_icu"] = length_of_stay_icu
normed_df["days_to_death"] = days_to_death
normed_df["died_in_hospital"] = died_in_hospital

normed_df.describe()


,age,gender,weight,height,heartrate_mean,heartrate_min,heartrate_max,meanbp_mean,meanbp_min,meanbp_max,...,1430,1431,1432,1433,1434,1435,1436,1437,length_of_stay_hospital,days_to_death
count,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,...,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,4.927600e+04,49276.000000,21266.000000
mean,3.785158e-17,2.667635e-18,-2.447736e-17,-8.201177e-19,4.217748e-18,-2.703685e-17,4.866632e-18,1.292361e-17,-5.767861e-18,5.302827e-17,...,-6.092303e-18,-7.966857e-18,9.805363e-18,-1.103103e-17,6.660076e-18,-8.651791e-18,7.678464e-18,-4.181699e-18,10.001152,516.919261
std,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,...,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,1.000010e+00,10.824429,750.269258
min,-2.851985e+00,-1.132784e+00,-3.679919e+00,-7.150134e+00,-3.551337e+00,-4.715110e+00,-3.556091e+00,-5.410569e+00,-4.293148e+00,-3.161874e+00,...,-1.044598e+01,-1.423567e+01,-1.254131e+01,-1.166184e+01,-8.771547e+00,-1.245394e+01,-1.092345e+01,-1.362189e+01,0.000000,0.000000
25%,-6.602848e-01,-1.132784e+00,-6.305664e-01,-1.308840e-17,-7.229681e-01,-6.636825e-01,-6.937438e-01,-7.131228e-01,-5.493726e-01,-5.718688e-01,...,-2.307914e-01,-2.201078e-01,-1.900663e-01,-1.945371e-01,-2.193400e-01,-1.964945e-01,-2.140415e-01,-2.077441e-01,4.040000,18.000000
50%,8.950739e-02,8.827810e-01,8.096634e-17,-1.308840e-17,-8.015685e-02,-6.248149e-02,-1.115714e-01,-9.270538e-02,1.349679e-16,-1.853008e-01,...,-2.542474e-02,-2.151379e-02,4.834280e-03,2.598291e-03,-9.215172e-03,2.073447e-04,-1.437202e-02,-1.388533e-02,6.920000,149.000000
75%,7.816233e-01,8.827810e-01,3.858846e-01,-1.308840e-17,6.269355e-01,6.055197e-01,6.161441e-01,5.277120e-01,5.517378e-01,3.172375e-01,...,1.692542e-01,1.802531e-01,1.904886e-01,1.836319e-01,1.849409e-01,2.104166e-01,1.817449e-01,1.797262e-01,11.950000,722.750000
max,1.531416e+00,8.827810e-01,1.013457e+01,3.581620e+01,5.062333e+00,4.680327e+00,8.572500e+00,6.731886e+00,4.882772e+00,7.507402e+00,...,1.253282e+01,1.584002e+01,1.445355e+01,1.794241e+01,1.372316e+01,1.258050e+01,1.579031e+01,1.538091e+01,294.660000,4166.000000


In [23]:
pk.dump(scaler, open(f"{df_path}/scaler_{df_name}.pkl", 'wb')) 

## Save DataFrame:

In [24]:
normed_df.to_pickle(f"{df_path}/dataframe_{df_name}.pkl")